In [1]:
import os
import yaml
import json
import pandas as pd

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
import wombat.bsub as bsub

load sample data

In [7]:
m = yaml.safe_load(open('../tests/data/cnv/C3L-00677.yaml'))
m

{'id': 'C3L-00677',
 'rna-seq': {'tumor': {'R1': {'filepath': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/test_samples/C3L-00677/rna-seq/170906_UNC32-K00270_0058_BHLJ7FBBXX_TGACCA_S4_L007_R1_001.fastq.gz'},
   'R2': {'filepath': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/test_samples/C3L-00677/rna-seq/170906_UNC32-K00270_0058_BHLJ7FBBXX_TGACCA_S4_L007_R2_001.fastq.gz'}}},
 'wxs': {'normal': {'barcode': 'CPT0071960002',
   'filepath': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/test_samples/C3L-00677/wxs/5cee818c-61e8-4441-bd9a-5c984cde9c6b_gdc_realn.bam'},
  'tumor': {'barcode': 'CPT0071100006',
   'filepath': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/test_samples/C3L-00677/wxs/1d6c4f0f-09c4-4472-a19e-30ee7e0ecf64_gdc_realn.bam'}}}

make test input yaml

In [9]:
template = yaml.safe_load(open('/home/estorrs/pecgs-pipeline/submodules/pecgs-cnv/cwl/template.cnv.yaml'))
template

{'common_biallelic': {'class': 'File', 'path': 'a/file/path'},
 'genome': {'class': 'File', 'path': 'a/file/path'},
 'genome_dict': {'class': 'File', 'path': 'a/file/path'},
 'normal_bam': {'class': 'File', 'path': 'a/file/path'},
 'pool_of_normals': {'class': 'File', 'path': 'a/file/path'},
 'protein_coding_gene': {'class': 'File', 'path': 'a/file/path'},
 'sample': 'a_string',
 'target_interval_list': {'class': 'File', 'path': 'a/file/path'},
 'tumor_bam': {'class': 'File', 'path': 'a/file/path'}}

In [10]:
template['common_biallelic']['path'] = '/scratch1/fs1/dinglab/estorrs/pecgs_resources/cnv/db/snp_database/af-only-gnomad.hg38.common_biallelic.chr1-22XY.vcf'
template['genome']['path'] = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.fa'
template['genome_dict']['path'] = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.dict'
template['pool_of_normals']['path'] = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/create_pon/gatk4scnaPON.Normal.hdf5'
template['protein_coding_gene']['path'] = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/db/remaking_protein-coding/gencode.v34.annotation.gene_filterd.need_gene_symbol.no_sym.filtered_to_hgnc_protein-coding_genes.bed'
template['target_interval_list']['path'] = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/db/IDT_xGen_Exome_Research_Panel_v1.hg38.removed_alt_chr.bed.target.preprocessed.exome.interval_list'


template['normal_bam']['path'] = m['wxs']['normal']['filepath']
template['tumor_bam']['path'] = m['wxs']['tumor']['filepath']
template['sample'] = m['id']




In [11]:
template

{'common_biallelic': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/db/snp_database/af-only-gnomad.hg38.common_biallelic.chr1-22XY.vcf'},
 'genome': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.fa'},
 'genome_dict': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/references/GRCh38.d1.vd1/GRCh38.d1.vd1.dict'},
 'normal_bam': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/test_samples/C3L-00677/wxs/5cee818c-61e8-4441-bd9a-5c984cde9c6b_gdc_realn.bam'},
 'pool_of_normals': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/create_pon/gatk4scnaPON.Normal.hdf5'},
 'protein_coding_gene': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/db/remaking_protein-coding/gencode.v34.an

In [12]:
yaml.safe_dump(template, open('../tests/data/cnv/C3L-00677_inputs.yaml', 'w'))

make cromwell compute1 config

In [13]:
workflow_root = '/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/cnv/C3L-00677'
output_fp = '../tests/data/cnv/C3L-00677.cromwell-config-db.compute1.dat'
bsub.save_compute1_cromwell_template(workflow_root, output_fp)

make execution script

In [14]:
TEST_DIR_ROOT = '/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/cnv'
TOOL_ROOT = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline'

dconfig = os.path.join(TEST_DIR_ROOT, 'C3L-00677.cromwell-config-db.compute1.dat')
cwl_fp = os.path.join(TOOL_ROOT, 'submodules', 'pecgs-cnv', 'cwl', 'cnv.cwl')
inputs_fp = os.path.join(TEST_DIR_ROOT, 'C3L-00677_inputs.yaml')
volumes = [TEST_DIR_ROOT, TOOL_ROOT, workflow_root, '/storage1/fs1/dinglab/Active', '/storage1/fs1/m.wyczalkowski/Active']

In [15]:
args = bsub.DEFAULT_ARGS
args['group_name'] = 'test_cromwell'
args

{'mem': 10,
 'max_mem': None,
 'docker': 'python:3.8',
 'queue': 'dinglab',
 'group': 'compute-dinglab',
 'group_name': 'test_cromwell',
 'n_concurrent': 10,
 'interactive': False,
 'username': 'estorrs'}

In [16]:
start_commands, run_command = bsub.cromwell_commands(dconfig, cwl_fp, inputs_fp, args, volumes)
filepath = os.path.join('../tests/data/cnv/start_server.sh')
bsub.write_command_file(start_commands, filepath)
filepath = os.path.join('../tests/data/cnv/run_job.sh')
bsub.write_command_file([run_command], filepath)

In [17]:
for c in start_commands: print(c)

source /opt/ibm/lsfsuite/lsf/conf/lsf.conf
export LSF_DOCKER_NETWORK=host
export LSF_DOCKER_VOLUMES="/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/cnv:/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/cnv /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline:/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/cnv/C3L-00677:/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/cnv/C3L-00677 /storage1/fs1/dinglab/Active:/storage1/fs1/dinglab/Active /storage1/fs1/m.wyczalkowski/Active:/storage1/fs1/m.wyczalkowski/Active"
bgadd -L 10 /estorrs/test_cromwell
bsub -q dinglab-interactive -G compute-dinglab -a 'docker(mwyczalkowski/cromwell-runner)' -Is '/bin/bash'


In [18]:
print(run_command)

/usr/bin/java -Dconfig.file=/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/cnv/C3L-00677.cromwell-config-db.compute1.dat -Djavax.net.ssl.trustStorePassword=changeit -Djavax.net.ssl.trustStore=/gscmnt/gc2560/core/genome/cromwell/cromwell.truststore -jar /usr/local/cromwell/cromwell-47.jar run -t cwl -i /storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/cnv/C3L-00677_inputs.yaml /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline/submodules/pecgs-cnv/cwl/cnv.cwl
